# Flexibility

In this section we'll be looking at ways of extending the functionality of Modern Fortran, using features widely used in other programming languages such as C++, but which have no equivalence in legacy Fortran.

## Derived Types

We've seen examples of _Intrinsic Types_, the variable types that can be used in Fortran code. We can imagine the situation where we have a very large number of variables decribing the property of an object, any of which are meaningless outside the context of the whole. Take a car, for example. We let if have `colour`, `doors`, `seats`, `age`, and a number of other attributes, but all of these refer to a single object, _the car_. To write any function or subroutine that performs an operation on _the car_, one must pass all its properties separately (and out of context) and return them. Particularly if there a very large number of attributes, this can make code unreadable or potentially even breach Fortran's limit on the permitted number of arguments.

Fortunately, there is an alternative. A _Derived Type_ is a user-defined compound variable consisting of a number of intrinsic types, and can be used to package related variables, giving them context. An example of a very simple derived type follows:

```fortran
Type <name>
  Type    :: var_name
  Type    :: var_name
End type <name>
```
This is simply a definition. We'd then have to declare an object of this type like so:

```fortran
Type (<name>)  :: my_object
```

We can also have more than one object of this type, or even an array of them.

For our car, we could define a type as follows:

In [ ]:
%num_images: 1
Program kind_demo
  Implicit none

  ! Define a derived type describing a car
  Type car
    Character(15)  :: colour
    Character(15)  :: name
    Integer        :: doors
    Integer        :: seats
    Integer        :: age
  End type car
  
  ! Declare an array of 'cars'
  Type (car)       :: some_cars(2)
  
  ! Declare a loop counter
  Integer          :: x
  
  ! Populate the first car one element at a time
  some_cars(1)%colour = 'Red'
  some_cars(1)%name   = 'My Car'
  some_cars(1)%doors  = 5
  some_cars(1)%seats  = 4
  some_cars(1)%age    = 3
  
  ! Populate the second car all at once
  some_cars(2) = car(colour='Blue', name='Some Other Car', doors=7, seats=2, age=5)
  
  ! Print out the details of each car
  do x = 1, 2
    print *, some_cars(x)%name
    print *, some_cars(x)%colour
    print "(i2)", some_cars(x)%doors
    print "(i2)", some_cars(x)%seats
    print "(i2)", some_cars(x)%age
  end do
  
End Program

Note that the syntax for accessing an element within a derived type is to prefix the name of the element with the name of the containing object separated by a `%` sign, i.e. `some_cars(1)%age`.

This `car` object can then be passed to functions, assuming that both program units are aware of the definition of the derived type. The ideal way to do this would be to place it in the header of a _module_ and `use` that module in any units that require the definition. An example of this is shown below:

| Program  | Directory              | Purpose                                                    |
| -------- | ---------------------- | ---------------------------------------------------------- |
| **cars** | `src/derived/example1` | Demonstrate how derived types can be used in modules       |

Note that because we are passing objects of type `car` into a subroutine with `intent(in)` we can't just update that object and return it. We have to make a working copy and update that. This is the case with all variables passed with this specific intent and means we are sacrificing some speed and memory for better quality control.

### Exercise 1 - _Derived Types_

 * Create your own derived type and 'process' it as in the above example.

## Dynamic Memory

In the examples so far we've specified the size of a required array in the program code. If we need an array of 15 integers we have to write it like this:

```fortran
Integer  :: my_array(15)
```

But what if we don't know how large an array we need? Let's say that we are reading in entries from a file and don't know ahead of time how many to expect. We could set a fixed upper limit (15 in this case) and risk either wasting memory or running out, but what we can't do is leave the length undefined and assume we increase it according to demand as we can in language like Python. We can't, for example, do this:

```fortran
Integer  :: my_array(*)
```

This is because the Fortran compiler needs to know the size of these variables at compile time in order to be able to allocate the space for them.

_However_, although we can't declare standard arrays of undefined size, we can use the `allocatable` attribute, to defer the choice of size from _compile time_ to _run time_. We could then test the number of elements we need to read it, perhaps by querying the size of a file, and then allocating enough memory to hold that data.

An two-dimensional 'allocatable' array is declared as follows:

```fortran
Integer, allocatable  :: my_array(:, :)
```

At this point the array has undefined size and can't be used. Once we know the amount of data that the array needs to hold we can allocate that space:

```fortran
var_x = 20
var_y = 15

! Allocate space to the reserved 'my_array'
allocate(my_array(var_x, var_y)
```

A working demonstration follows. Since allocating an array twice without first `deallocating` it is not allowed, we can use the `allocated()` intrinsic to check its status before allocating it some memory.

In [ ]:
%num_images: 1
Program dynamic_demo
  Implicit none
  
  ! Declare an allocatable array of currently undefined size
  Integer, allocatable :: my_array(:, :)
  
  ! Decare variables to hold the size of the array dimensions and indices
  Integer              :: x_size, y_size
  Integer              :: x, y
  
  ! Imagine we have determined the required extent of our array
  x_size = 15
  y_size = 7
  
  ! If my_array has not yet been allocated then assign it some storage space
  if (.not. allocated(my_array)) then
    allocate(my_array(x_size, y_size))
  else
    print *, 'my_array already has been allocated'
  end if  
  
  ! Check the size of each dimension
  print *, 'Size of first dimension:  ', size(my_array, 1)
  print *, 'Size of second dimension: ', size(my_array, 2)
  
  ! Try allocating it again. Fortran doesn't allow this.
  if (.not. allocated(my_array)) then
    allocate(my_array(x_size, y_size))
  else
    print *, 'my_array already has been allocated'
  end if  

End Program

Once allocated, _allocatable arrays_ can be treated exactly like standard arrays. They have _interfaces_ automatically generated when used in a module, and they can even be returned from subroutines and functions, as shown in the following example:

| Program     | Directory              | Purpose                                                    |
| ----------- | ---------------------- | ---------------------------------------------------------- |
| **dynamic** | `src/dynamic/example1` | Example of how dynamic arrays can be passed between units  |

In this case the subroutine doesn't need to know what size of array it has received. All features of the array can be determined by using intrinsic functions, so we don't have to modify the code depending on the size of the array received. Note that even though the dynamic array is allocated in the subroutine, it stays in scope (remains allocated) along with its values, when returned back to the main program.

### Excercise 2 - _Dynamic Memory_

 * The scope of dynamic arrays in modules is a little counterintuitive. We can declare a dynamic array in the header (before `contains`) and allocate it in the subroutine, as though we were treating the module like a _Class_. If we do that the the variable remains in scope as long as the module is loaded (i.e. forever!). Try doing this and calling `proc` repeatedly. How would you solve this problem?

## Parallelising your Code

The topic of parallelising code is a very large one and will not be dealt with in great depth here. Fortran compilers generally support _OpenMP_, and there are a large number of _Message Passing_ libraries available, such as a Intel MPI and the open source _OpenMPI_ (not to be confused with OpenMP).

In addition to these general protocols, Fortran offers a specific method of parallel programming called **CoArrays**. It has similarities to _MPI_ but automates a lot of the work involved in MPI programming and provides functions for dealing with parallelised data that would be more familiar to the Fortran programmer.

### CoArray Programming

A CoArray Fortran program consists of a number of identical images, or copies of the program, running simultaneously and independently. Like _MPI_, it is possible to query the numerical ID of the image to determine one from another, and explicitly pass messages from one to another (point-to-point) or across all processes (collective). In this way, a large dataset can be split up into chunks, with each chunk belonging to a separate process. Unlike MPI, CoArray allows each processes to directly access the data of another process without explicit message passing.

A simple example, demonstrating the MPI-like ability to query the ID of a process is shown below. This uses the Coarray functions `this_image()` and `num_images()` to identify an individual process within the total pool. 

In [ ]:
%num_images: 4
Program query_id
  Implicit none

  ! Print the image or process ID, and the total number of images
  print *, 'I am process number: ', this_image(), ' out of ', num_images()
End program

#### Arrays

The heart of CoArrays is the concept that each process has it's own copy of an array, but can also access data belonging to the array of the same name from any other process. Accessing the local copy of an array is identical to standard Fortran syntax:

```fortran
  my_array(2) = 100
```

This simply assigns the value 100 to element 2 of a one dimensional array named `my_array`.

Accessing a copy of the array belonging to another process is achieved by including the ID of that process in `[]` brackets:

```fortran
  my_array[2](2) = 120
```

Assuming that the process ID is 1, then this operation will assign the value of 120 to the copy of `my_array` belonging to process 2. Behind the scenes this is achieved by an MPI _point-to-point_ communication.

A functional example of this using two processes is shown below:

In [ ]:
%num_images: 2
Program coarray
  Implicit none

  ! Declare a one-dimensional 'coarray' with four elements. The `[*]` is known as the co-index
  ! and should generally not be changed. It is assigned with the program is run and is equal
  ! to the number of processes.
  Integer :: my_array(4)[*]
  
  ! Array index counter
  Integer :: x
  
  ! Each process can assign values to its own copy of the array.
  do x = 1, ubound(my_array, 1)
    my_array(x) = this_image() * 100 + x
  end do
  
  ! Print out the coarray for each image
  print "(A6x, i1, A7x, 4i6)", 'image:', this_image(), 'array:', my_array
  
  ! Image 1 can change values in Image 2's copy of my_array using the [x] image specifier
  if (this_image() == 1) then
    my_array(3)[2] = 555
  end if
  
  ! Since images run indendently we need to make sure the above operation completed
  ! before we try and test the results
  sync all
  
  ! Print out the coarrays again to check this worked
  print "(A6x, i1, A7x, 4i6)", 'image:', this_image(), 'array:', my_array

End program

### Exercise 3 - _Arrays_

 * What happens if you remove the `sync` line from the above example. Try running the code multiple times with sync removed and see if you can explain the behaviour. Remember, images will run independently unless communication takes place between them.
 * Can you change the example so that image 2 _also_ changes the values in image 1?